# CubeMaster: Color Classification Model Training

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mvipin/cubemaster/blob/main/notebooks/colab_training.ipynb)

This notebook enables training of **MLP** and **Shallow CNN** models for Rubik's Cube color classification using Google Colab's GPU resources.

## Contents
1. [Environment Setup](#1-environment-setup)
2. [Data Preparation](#2-data-preparation)
3. [Model Training](#3-model-training)
4. [Weights & Biases Integration](#4-weights--biases-integration)
5. [Results Visualization](#5-results-visualization)
6. [Model Export](#6-model-export)

---
## 1. Environment Setup

### 1.1 Check GPU Availability

In [ ]:
# Check GPU availability
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU Available: {gpu_name}")
    print(f"   Memory: {gpu_memory:.1f} GB")
else:
    print("⚠️ No GPU detected. Training will be slower.")
    print("   Go to Runtime > Change runtime type > Hardware accelerator > GPU")

### 1.2 Install Dependencies

In [ ]:
# Install required packages
!pip install -q torch torchvision torchmetrics
!pip install -q albumentations opencv-python-headless Pillow
!pip install -q numpy pyyaml tqdm
!pip install -q matplotlib seaborn scikit-learn
!pip install -q onnx onnxruntime
!pip install -q wandb

print("✅ All dependencies installed!")

### 1.3 Clone Repository

In [ ]:
import os
from pathlib import Path

# Install Git LFS (required for dataset files)
!apt-get install -qq git-lfs
!git lfs install

# Clone the repository
REPO_URL = "https://github.com/mvipin/cubemaster.git"
REPO_DIR = Path("/content/cubemaster")

if not REPO_DIR.exists():
    !git clone {REPO_URL} {REPO_DIR}
    print(f"✅ Repository cloned to {REPO_DIR}")
else:
    print(f"✅ Repository already exists at {REPO_DIR}")
    # Pull latest changes
    !cd {REPO_DIR} && git pull

# Pull LFS files (dataset images)
print("\n📥 Pulling Git LFS files (dataset images)...")
!cd {REPO_DIR} && git lfs pull
print("✅ LFS files downloaded")

# Change to project directory
os.chdir(REPO_DIR)
print(f"📁 Working directory: {os.getcwd()}")

### 1.4 Setup Python Path

In [ ]:
import sys
from pathlib import Path

# Add src to Python path
SRC_PATH = Path("/content/cubemaster/src")
if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

# Verify imports work
try:
    from cubemaster.models import MLPClassifier, ShallowCNNClassifier, MODEL_REGISTRY
    from cubemaster.utils.config import load_config, get_device, set_seed
    from cubemaster.training.dataset import CubeColorDataset
    from cubemaster.training.augmentations import get_train_transforms, get_val_transforms
    from cubemaster.training.trainer import Trainer, EarlyStopping
    from cubemaster import COLOR_CLASSES
    print("✅ All imports successful!")
    print(f"   Available models: {list(MODEL_REGISTRY.keys())}")
    print(f"   Color classes: {COLOR_CLASSES}")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("   Make sure the repository is cloned correctly.")

---
## 2. Data Preparation

### 2.1 Dataset Selection

The CubeMaster repository includes a pre-processed dataset with 4,410 images ready for training.

**Dataset options:**
1. **Repository Dataset** (default) - Use the included `data/processed/` dataset
2. **Google Drive** - Upload your own dataset via Google Drive
3. **Direct Upload** - Upload a zip file directly

In [ ]:
from pathlib import Path

# Dataset source selection
# Options: 'repository', 'google_drive', 'upload'
DATASET_SOURCE = 'repository'  # Default: use included dataset

if DATASET_SOURCE == 'repository':
    # Use the dataset included in the cloned repository
    DATASET_PATH = Path("/content/cubemaster/data/processed")
    
    if DATASET_PATH.exists():
        print(f"✅ Using repository dataset: {DATASET_PATH}")
        print("   This dataset contains 4,410 images (train: 4180, val: 101, test: 129)")
    else:
        print(f"❌ Repository dataset not found at {DATASET_PATH}")
        print("   Make sure the repository was cloned correctly.")

elif DATASET_SOURCE == 'google_drive':
    # Mount Google Drive and use custom dataset
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Set path to your dataset in Google Drive
    # Expected structure: dataset/train/{B,G,O,R,W,Y}/*.jpg
    DATASET_PATH = Path("/content/drive/MyDrive/CubeMaster/dataset")
    print(f"📁 Using Google Drive dataset: {DATASET_PATH}")

elif DATASET_SOURCE == 'upload':
    # Direct upload via Colab file picker
    from google.colab import files
    import zipfile
    
    print("Upload your dataset as a zip file:")
    print("Expected structure: train/{B,G,O,R,W,Y}/*.jpg")
    uploaded = files.upload()
    
    # Extract uploaded zip
    for filename in uploaded.keys():
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('/content/dataset')
    DATASET_PATH = Path("/content/dataset")
    print(f"✅ Dataset extracted to {DATASET_PATH}")

else:
    raise ValueError(f"Unknown DATASET_SOURCE: {DATASET_SOURCE}")

### 2.2 Verify Dataset Structure

In [ ]:
import os
from pathlib import Path
from collections import defaultdict

def verify_dataset(dataset_path: Path):
    """Verify dataset structure and count samples."""
    dataset_path = Path(dataset_path)
    
    if not dataset_path.exists():
        print(f"❌ Dataset path does not exist: {dataset_path}")
        return None
    
    splits = ['train', 'val', 'test']
    expected_classes = ['B', 'G', 'O', 'R', 'W', 'Y']
    stats = defaultdict(dict)
    
    print(f"📊 Dataset Statistics for: {dataset_path}\n")
    print(f"{'Split':<10} {'B':>6} {'G':>6} {'O':>6} {'R':>6} {'W':>6} {'Y':>6} {'Total':>8}")
    print("-" * 60)
    
    for split in splits:
        split_path = dataset_path / split
        if not split_path.exists():
            print(f"⚠️  {split:<10} - Directory not found")
            continue
        
        total = 0
        row = f"{split:<10}"
        for cls in expected_classes:
            cls_path = split_path / cls
            if cls_path.exists():
                count = len(list(cls_path.glob('*.png'))) + len(list(cls_path.glob('*.jpg')))
                stats[split][cls] = count
                total += count
                row += f" {count:>6}"
            else:
                row += f" {'N/A':>6}"
        stats[split]['total'] = total
        row += f" {total:>8}"
        print(row)
    
    print("-" * 60)
    return stats

# Verify dataset
dataset_stats = verify_dataset(DATASET_PATH)

### 2.3 Setup Data Loaders

In [ ]:
from torch.utils.data import DataLoader
from cubemaster.training.dataset import CubeColorDataset
from cubemaster.training.augmentations import get_train_transforms, get_val_transforms

def create_data_loaders(dataset_path, batch_size=32, num_workers=2, image_size=(50, 50)):
    """Create train, validation, and test data loaders."""
    dataset_path = Path(dataset_path)
    
    # Get transforms
    train_transform = get_train_transforms(image_size)
    val_transform = get_val_transforms(image_size)
    
    # Create datasets
    train_dataset = CubeColorDataset(
        root_dir=dataset_path / 'train',
        transform=train_transform
    )
    val_dataset = CubeColorDataset(
        root_dir=dataset_path / 'val',
        transform=val_transform
    )
    test_dataset = CubeColorDataset(
        root_dir=dataset_path / 'test',
        transform=val_transform
    )
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True,
        num_workers=num_workers, pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=True
    )
    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=True
    )
    
    print(f"✅ Data loaders created:")
    print(f"   Train: {len(train_dataset)} samples, {len(train_loader)} batches")
    print(f"   Val:   {len(val_dataset)} samples, {len(val_loader)} batches")
    print(f"   Test:  {len(test_dataset)} samples, {len(test_loader)} batches")
    
    return train_loader, val_loader, test_loader

# Create data loaders (will be called after config is set)
print("Data loader function defined. Will be created during training.")

---
## 3. Model Training

### 3.1 Training Configuration

In [ ]:
import yaml
from cubemaster.utils.config import load_config, set_seed

# Training configuration
TRAINING_CONFIG = {
    # Model selection: 'mlp' or 'shallow_cnn'
    'model_type': 'mlp',  # Change to 'shallow_cnn' for CNN training
    
    # Training hyperparameters
    'batch_size': 32,
    'epochs': 50,
    'learning_rate': 0.001,
    'weight_decay': 1e-4,
    
    # MLP-specific
    'mlp_hidden_dims': [256, 128],
    'mlp_dropout': 0.3,
    
    # Shallow CNN-specific
    'cnn_dropout': 0.5,
    
    # Early stopping
    'patience': 10,
    'min_delta': 0.001,
    
    # Reproducibility
    'seed': 42,
    
    # Wandb (optional)
    'use_wandb': False,  # Set to True to enable wandb logging
    'wandb_project': 'cubemaster',
}

# Set seed for reproducibility
set_seed(TRAINING_CONFIG['seed'])
print(f"✅ Configuration set for {TRAINING_CONFIG['model_type'].upper()} training")
print(f"   Epochs: {TRAINING_CONFIG['epochs']}, Batch size: {TRAINING_CONFIG['batch_size']}")
print(f"   Learning rate: {TRAINING_CONFIG['learning_rate']}")

### 3.2 Train MLP Model

In [ ]:
import torch
import torch.nn as nn
from cubemaster.models import MLPClassifier
from cubemaster.training.trainer import Trainer, EarlyStopping
from cubemaster.utils.config import get_device

def train_mlp(config, train_loader, val_loader):
    """Train MLP model with given configuration."""
    device = get_device()
    print(f"\n🚀 Training MLP on {device}")
    print("=" * 60)
    
    # Create model
    model = MLPClassifier(
        num_classes=6,
        input_size=(50, 50),
        hidden_dims=config['mlp_hidden_dims'],
        dropout_rate=config['mlp_dropout']
    ).to(device)
    
    params = model.count_parameters()
    print(f"Model parameters: {params['trainable']:,} trainable")
    
    # Setup training components
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=config['learning_rate'],
        weight_decay=config['weight_decay']
    )
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=5
    )
    early_stopping = EarlyStopping(
        patience=config['patience'],
        min_delta=config['min_delta']
    )
    
    # Create trainer
    trainer = Trainer(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        early_stopping=early_stopping,
        checkpoint_dir=Path('/content/cubemaster/models/mlp'),
        use_wandb=config['use_wandb']
    )
    
    # Train
    history = trainer.fit(train_loader, val_loader, epochs=config['epochs'])
    
    print("=" * 60)
    print(f"✅ Training complete! Best val accuracy: {trainer.best_val_acc:.2f}%")
    
    return model, history, trainer

# Train MLP (set model_type='mlp' in config above)
if TRAINING_CONFIG['model_type'] == 'mlp':
    train_loader, val_loader, test_loader = create_data_loaders(
        DATASET_PATH, 
        batch_size=TRAINING_CONFIG['batch_size']
    )
    mlp_model, mlp_history, mlp_trainer = train_mlp(TRAINING_CONFIG, train_loader, val_loader)
else:
    print("ℹ️ Skipping MLP training. Set model_type='mlp' to train.")

### 3.3 Train Shallow CNN Model

In [ ]:
from cubemaster.models import ShallowCNNClassifier

def train_shallow_cnn(config, train_loader, val_loader):
    """Train Shallow CNN model with given configuration."""
    device = get_device()
    print(f"\n🚀 Training Shallow CNN on {device}")
    print("=" * 60)
    
    # Create model
    model = ShallowCNNClassifier(
        num_classes=6,
        input_size=(50, 50),
        dropout_rate=config['cnn_dropout']
    ).to(device)
    
    params = model.count_parameters()
    print(f"Model parameters: {params['trainable']:,} trainable")
    
    # Setup training components
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=config['learning_rate'],
        weight_decay=config['weight_decay']
    )
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=5
    )
    early_stopping = EarlyStopping(
        patience=config['patience'],
        min_delta=config['min_delta']
    )
    
    # Create trainer
    trainer = Trainer(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        early_stopping=early_stopping,
        checkpoint_dir=Path('/content/cubemaster/models/shallow_cnn'),
        use_wandb=config['use_wandb']
    )
    
    # Train
    history = trainer.fit(train_loader, val_loader, epochs=config['epochs'])
    
    print("=" * 60)
    print(f"✅ Training complete! Best val accuracy: {trainer.best_val_acc:.2f}%")
    
    return model, history, trainer

# Train Shallow CNN (set model_type='shallow_cnn' in config above)
if TRAINING_CONFIG['model_type'] == 'shallow_cnn':
    train_loader, val_loader, test_loader = create_data_loaders(
        DATASET_PATH, 
        batch_size=TRAINING_CONFIG['batch_size']
    )
    cnn_model, cnn_history, cnn_trainer = train_shallow_cnn(TRAINING_CONFIG, train_loader, val_loader)
else:
    print("ℹ️ Skipping Shallow CNN training. Set model_type='shallow_cnn' to train.")

---
## 4. Weights & Biases Integration

### 4.1 Wandb Authentication

In [ ]:
# Wandb setup for Colab
import wandb

# Login to wandb (will prompt for API key)
# Get your API key from: https://wandb.ai/authorize
wandb.login()

print("✅ Logged into Weights & Biases!")
print("   Dashboard: https://wandb.ai/home")

### 4.2 Train with Wandb Logging

In [ ]:
# Enable wandb and retrain
TRAINING_CONFIG['use_wandb'] = True

# Initialize wandb run
run = wandb.init(
    project=TRAINING_CONFIG['wandb_project'],
    name=f"{TRAINING_CONFIG['model_type']}_colab_{TRAINING_CONFIG['seed']}",
    config=TRAINING_CONFIG,
    tags=['colab', TRAINING_CONFIG['model_type']]
)

print(f"📊 Wandb run initialized: {run.url}")

# Create data loaders
train_loader, val_loader, test_loader = create_data_loaders(
    DATASET_PATH, 
    batch_size=TRAINING_CONFIG['batch_size']
)

# Train selected model with wandb
if TRAINING_CONFIG['model_type'] == 'mlp':
    model, history, trainer = train_mlp(TRAINING_CONFIG, train_loader, val_loader)
else:
    model, history, trainer = train_shallow_cnn(TRAINING_CONFIG, train_loader, val_loader)

# Finish wandb run
wandb.finish()
print(f"\n✅ Training logged to wandb: {run.url}")

---
## 5. Results Visualization

### 5.1 Training Curves

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_training_curves(history, title="Training Curves"):
    """Plot training and validation curves."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    epochs = range(1, len(history['train_loss']) + 1)
    
    # Loss plot
    axes[0].plot(epochs, history['train_loss'], 'b-', label='Train Loss', linewidth=2)
    axes[0].plot(epochs, history['val_loss'], 'r-', label='Val Loss', linewidth=2)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Loss Curves')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Accuracy plot
    axes[1].plot(epochs, history['train_acc'], 'b-', label='Train Acc', linewidth=2)
    axes[1].plot(epochs, history['val_acc'], 'r-', label='Val Acc', linewidth=2)
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy (%)')
    axes[1].set_title('Accuracy Curves')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Mark best epoch
    best_epoch = np.argmax(history['val_acc']) + 1
    best_acc = max(history['val_acc'])
    axes[1].axvline(x=best_epoch, color='g', linestyle='--', alpha=0.5, label=f'Best: {best_acc:.1f}%')
    axes[1].scatter([best_epoch], [best_acc], color='g', s=100, zorder=5)
    
    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()
    
    return fig

# Plot training curves
if 'history' in dir():
    plot_training_curves(history, f"{TRAINING_CONFIG['model_type'].upper()} Training Curves")
else:
    print("ℹ️ No training history available. Run training first.")

### 5.2 Confusion Matrix

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from cubemaster import COLOR_CLASSES

def evaluate_and_plot_confusion_matrix(model, test_loader, device):
    """Evaluate model and plot confusion matrix."""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())
    
    # Calculate accuracy
    accuracy = 100 * sum(p == l for p, l in zip(all_preds, all_labels)) / len(all_labels)
    print(f"\n📊 Test Accuracy: {accuracy:.2f}%\n")
    
    # Classification report
    print("Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=COLOR_CLASSES))
    
    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=COLOR_CLASSES, yticklabels=COLOR_CLASSES, ax=ax)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('True')
    ax.set_title(f'Confusion Matrix (Accuracy: {accuracy:.2f}%)')
    plt.tight_layout()
    plt.show()
    
    return accuracy, cm

# Evaluate on test set
if 'model' in dir() and 'test_loader' in dir():
    device = get_device()
    test_accuracy, cm = evaluate_and_plot_confusion_matrix(model, test_loader, device)
else:
    print("ℹ️ No model or test data available. Run training first.")

### 5.3 Model Comparison (Optional)

Train both models and compare their performance.

In [ ]:
def compare_models(results_dict):
    """Compare multiple models side by side."""
    if len(results_dict) < 2:
        print("ℹ️ Need at least 2 models to compare.")
        return
    
    # Create comparison table
    print("\n" + "=" * 60)
    print("📊 MODEL COMPARISON")
    print("=" * 60)
    print(f"{'Model':<20} {'Test Acc':>12} {'Parameters':>15}")
    print("-" * 60)
    
    for name, data in results_dict.items():
        acc = data.get('accuracy', 'N/A')
        params = data.get('params', 'N/A')
        print(f"{name:<20} {acc:>11.2f}% {params:>15,}")
    
    print("=" * 60)
    
    # Bar chart comparison
    fig, ax = plt.subplots(figsize=(10, 5))
    models = list(results_dict.keys())
    accuracies = [results_dict[m]['accuracy'] for m in models]
    
    bars = ax.bar(models, accuracies, color=['#2196F3', '#4CAF50'][:len(models)])
    ax.set_ylabel('Test Accuracy (%)')
    ax.set_title('Model Comparison')
    ax.set_ylim(0, 100)
    
    for bar, acc in zip(bars, accuracies):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{acc:.1f}%', ha='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# Example usage (uncomment after training both models)
# comparison_results = {
#     'MLP': {'accuracy': mlp_test_accuracy, 'params': mlp_model.count_parameters()['trainable']},
#     'Shallow CNN': {'accuracy': cnn_test_accuracy, 'params': cnn_model.count_parameters()['trainable']}
# }
# compare_models(comparison_results)
print("ℹ️ Train both MLP and Shallow CNN to enable comparison.")

---
## 6. Model Export

### 6.1 Save PyTorch Model

In [ ]:
import os
from pathlib import Path

def save_pytorch_model(model, trainer, model_name, output_dir='/content/exported_models'):
    """Save trained model in PyTorch format."""
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Save full checkpoint (includes optimizer state, epoch, etc.)
    checkpoint_path = output_dir / f"{model_name}_checkpoint.pth"
    torch.save({
        'model_state_dict': model.state_dict(),
        'model_config': model.get_config(),
        'best_val_acc': trainer.best_val_acc,
        'epoch': trainer.current_epoch,
    }, checkpoint_path)
    print(f"✅ Checkpoint saved: {checkpoint_path}")
    
    # Save model weights only (smaller file)
    weights_path = output_dir / f"{model_name}_weights.pth"
    torch.save(model.state_dict(), weights_path)
    print(f"✅ Weights saved: {weights_path}")
    
    return checkpoint_path, weights_path

# Save trained model
if 'model' in dir() and 'trainer' in dir():
    model_name = TRAINING_CONFIG['model_type']
    checkpoint_path, weights_path = save_pytorch_model(model, trainer, model_name)
else:
    print("ℹ️ No model available. Run training first.")

### 6.2 Export to ONNX Format

In [ ]:
import onnx
import onnxruntime as ort

def export_to_onnx(model, model_name, output_dir='/content/exported_models', input_size=(50, 50)):
    """Export model to ONNX format."""
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    onnx_path = output_dir / f"{model_name}.onnx"
    
    # Set model to eval mode
    model.eval()
    model.cpu()
    
    # Create dummy input
    dummy_input = torch.randn(1, 3, input_size[0], input_size[1])
    
    # Export to ONNX
    print(f"\n📦 Exporting to ONNX...")
    torch.onnx.export(
        model,
        dummy_input,
        str(onnx_path),
        export_params=True,
        opset_version=11,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={
            'input': {0: 'batch_size'},
            'output': {0: 'batch_size'}
        },
        dynamo=False  # Use legacy exporter for compatibility
    )
    
    # Verify ONNX model
    onnx_model = onnx.load(str(onnx_path))
    onnx.checker.check_model(onnx_model)
    print(f"✅ ONNX model exported and verified: {onnx_path}")
    
    # Test with ONNX Runtime
    ort_session = ort.InferenceSession(str(onnx_path))
    ort_inputs = {ort_session.get_inputs()[0].name: dummy_input.numpy()}
    ort_outputs = ort_session.run(None, ort_inputs)
    print(f"   Output shape: {ort_outputs[0].shape}")
    
    # Get file size
    size_mb = os.path.getsize(onnx_path) / (1024 * 1024)
    print(f"   File size: {size_mb:.2f} MB")
    
    return onnx_path

# Export to ONNX
if 'model' in dir():
    model_name = TRAINING_CONFIG['model_type']
    onnx_path = export_to_onnx(model, model_name)
else:
    print("ℹ️ No model available. Run training first.")

### 6.3 Download Trained Models

In [ ]:
from google.colab import files
import os

def download_models(output_dir='/content/exported_models'):
    """Download all exported models."""
    output_dir = Path(output_dir)
    
    if not output_dir.exists():
        print("❌ No exported models found. Export models first.")
        return
    
    print("\n📥 Available models for download:")
    for f in output_dir.iterdir():
        size_mb = os.path.getsize(f) / (1024 * 1024)
        print(f"   - {f.name} ({size_mb:.2f} MB)")
    
    # Create zip archive
    import shutil
    zip_path = '/content/cubemaster_models.zip'
    shutil.make_archive('/content/cubemaster_models', 'zip', output_dir)
    print(f"\n📦 Created archive: {zip_path}")
    
    # Download
    print("\n⬇️ Starting download...")
    files.download(zip_path)

# Download models
download_models()

### 6.4 Copy to Google Drive (Optional)

In [ ]:
import shutil

def copy_to_drive(source_dir='/content/exported_models', drive_dest='/content/drive/MyDrive/CubeMaster/trained_models'):
    """Copy exported models to Google Drive."""
    source_dir = Path(source_dir)
    drive_dest = Path(drive_dest)
    
    if not source_dir.exists():
        print("❌ No exported models found.")
        return
    
    # Create destination directory
    drive_dest.mkdir(parents=True, exist_ok=True)
    
    # Copy files
    print(f"\n📁 Copying models to Google Drive...")
    for f in source_dir.iterdir():
        dest_file = drive_dest / f.name
        shutil.copy2(f, dest_file)
        print(f"   ✅ {f.name} -> {dest_file}")
    
    print(f"\n✅ All models copied to: {drive_dest}")

# Copy to Drive (uncomment to run)
# copy_to_drive()

---
## Troubleshooting

### Common Issues

**1. GPU Not Available**
- Go to Runtime > Change runtime type > Hardware accelerator > GPU
- If GPU quota exceeded, try later or use CPU (slower)

**2. Session Timeout**
- Colab sessions timeout after ~90 minutes of inactivity
- Enable wandb to save training progress
- Save checkpoints to Google Drive periodically

**3. Out of Memory**
- Reduce batch size
- Use a simpler model
- Clear memory: `torch.cuda.empty_cache()`

**4. Dataset Upload Issues**
- Use Google Drive for large datasets (>100MB)
- Compress dataset as ZIP before uploading
- Check file paths after extraction

In [ ]:
# Utility: Clear GPU memory
def clear_gpu_memory():
    """Clear GPU memory cache."""
    import gc
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("✅ GPU memory cleared")
        print(f"   Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
        print(f"   Cached: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

# Uncomment to clear memory
# clear_gpu_memory()